In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor
import requests as r
import pandas_datareader as pdr
from dotenv import load_dotenv
import os
load_dotenv()
token = os.getenv("ALPHA")
from time import sleep

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2022-05-13 2022-05-14


In [5]:
market.connect()
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
        test = pdr.yahoo.daily.YahooDailyReader(symbols=ticker
                                                , start=start
                                                , end=end
                                                , retry_count=3, pause=0.1, session=None, adjust_price=True)
        panda_reader_data = test.read()
        panda_reader_data["ticker"] = ticker
        panda_reader_data.reset_index(inplace=True)
        market.store("pdr_prices",panda_reader_data)
        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY'
        stuff = r.get(url,params={"apikey":token,"symbol":ticker,"outputsize":"compact"})
        data = stuff.json()
        example = pd.DataFrame.from_records(data["Time Series (Daily)"]).T
        for col in example.columns:
            example.rename(columns={col:col.split(".")[1].strip()},inplace=True)
        example.reset_index(inplace=True)
        example.rename(columns={"index":"date"},inplace=True)
        example["ticker"] = ticker
        market.store("alpha_prices",example)
        sleep(12)
    except Exception as e:
        print(str(e))
market.disconnect()

  1%|█▋                                                                                                                                                                      | 5/505 [01:01<1:24:24, 10.13s/it]

'Time Series (Daily)'


  1%|█▉                                                                                                                                                                      | 6/505 [01:03<1:01:24,  7.38s/it]

'Time Series (Daily)'


  1%|██▎                                                                                                                                                                       | 7/505 [01:05<46:41,  5.63s/it]

...

'Time Series (Daily)'


 64%|███████████████████████████████████████████████████████████████████████████████████████████████████████████                                                             | 322/505 [21:31<11:30,  3.77s/it]

'Time Series (Daily)'


 64%|███████████████████████████████████████████████████████████████████████████████████████████████████████████                                                             | 322/505 [21:35<12:16,  4.02s/it]


KeyboardInterrupt: 

In [ ]:
analysis = []
market.connect()
periods = ["year","quarter","month","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("prices",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

In [ ]:
period = "day"
a = pd.DataFrame(analysis)
a.sort_values(f"{period}_gain",ascending=False).head(50)

In [ ]:
a = a.merge(sp5[["Symbol","GICS Sector"]].rename(columns={"Symbol":"ticker"}),on="ticker")
a

In [ ]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)